# Machine Listening (Fall 2020)
# Lab8_2 : Music Composer

# 라이브러리 가져오기

In [15]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [18]:
tf.config.list_physical_devices('GPU')

[]

# 구글 드라이브 연동
학습 데이터를 가져오기 위해 구글 드라이브와 연동합니다.

링크를 클릭한 후 인증 코드를 복사하여 입력합니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
location = './'

# 데이터 전처리


미디 데이터에서 music21 라이브러리를 사용해 노트 데이터 불러오기

In [3]:
def get_notes():
    notes = []

    for file in glob.glob(location + "midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(location + 'data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


notes = get_notes()
n_vocab = len(set(notes))


Parsing ./midi_songs/Ff7-One_Winged.mid
Parsing ./midi_songs/sera_.mid
Parsing ./midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing ./midi_songs/braska.mid
Parsing ./midi_songs/BlueStone_LastDungeon.mid
Parsing ./midi_songs/DOS.mid
Parsing ./midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing ./midi_songs/fortresscondor.mid
Parsing ./midi_songs/ff6shap.mid
Parsing ./midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing ./midi_songs/sobf.mid
Parsing ./midi_songs/z_aeristhemepiano.mid
Parsing ./midi_songs/Eternal_Harvest.mid
Parsing ./midi_songs/ff4_piano_collections-main_theme.mid
Parsing ./midi_songs/ultimafro.mid
Parsing ./midi_songs/ff4-airship.mid
Parsing ./midi_songs/relmstheme-piano.mid
Parsing ./midi_songs/ff8-lfp.mid
Parsing ./midi_songs/thenightmarebegins.mid
Parsing ./midi_songs/FFIXQuMarshP.mid
Parsing ./midi_songs/ff7-mainmidi.mid
Parsing ./midi_songs/lurk_in_dark.mid
Parsing ./midi_songs/cosmo.mid
Parsing ./midi_songs/tpirtsd-piano.mid
Parsing ./midi

불러온 노트 데이터를 lstm 모델의 입력 sequence로 변환하기

100개의 노트를 입력하면 다음 노트를 예측하도록 입력 크기 100, 출력 크기 1의 형태로 변환합니다.

In [4]:
notes

['9.2',
 '0.5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 '9.2',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 '9.2',
 'F5',
 'G5',
 'A5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'C6',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'G5',
 '9.2',
 '9.2',
 '7.0',
 'A5',
 '5.10',
 '9.2',
 '5.10',
 '9.2',
 'G5',
 '5.10',
 '9.2',
 'F5',
 '5.10',
 '9.2',
 'D5',
 '5.10',
 'F5',
 'F5',
 'G5',
 '5.10',
 '7.0',
 'A5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'C6',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'G5',
 '9.2',
 '2.5.9',
 '0.4.7',
 'D5',
 '5.10',
 '9.2',
 '5.10',
 '9.2',
 '5.10',
 '9.2',
 '5.10',
 '9.2',
 '5.10',
 'F5',
 'G5',
 '5.10',
 '7.0',
 'A5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'C6',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'G5',
 '9.2',
 '9.2',
 '7.0',
 'A5',
 '5.10',
 '9.2',
 '5.10',
 '9.2',
 'G5',
 '5.10',
 '9.2',
 'F5',
 '5.10',
 '9.2',
 'D5',
 '5.10',
 'F5',
 'F5',
 'G5',
 '5.10',
 '7.0',
 'A5',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'C6',
 '9.2',
 '0.5',
 '9.2',
 '0.5',
 'E6',
 '9.2',
 'C6',
 '9.2',
 '

In [5]:
n_vocab

158

In [6]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

network_input, network_output = prepare_sequences(notes, n_vocab)


In [7]:
network_input[0].shape

(100, 1)

In [8]:
network_input.shape

(5399, 100, 1)

In [10]:
N 5399 time 100 channel 1

SyntaxError: invalid syntax (<ipython-input-10-e75977616a86>, line 1)

# 모델 정의하기

keras를 사용해서 lstm 모델을 설계하고 정의합니다.

In [11]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


model = create_network(network_input, n_vocab)


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 512)          1052672   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
activation (Activation)      (None, 256)               0

# 모델 학습하기

In [19]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = location + "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)

train(model, network_input, network_output)

Epoch 1/50
43/43 [==============================] - 91s 2s/step - loss: 4.9376
Epoch 2/50
43/43 [==============================] - 90s 2s/step - loss: 4.7395
Epoch 3/50
24/43 [===============>..............] - ETA: 39s - loss: 4.6097

KeyboardInterrupt: 

In [ ]:
# network_input, network_output = prepare_sequences(notes, n_vocab)

# model = create_network(network_input, n_vocab)

# train(model, network_input, network_output)


Epoch 1/200
57077/57077 [==============================] - 234s 4ms/step - loss: 5.2028
Epoch 2/200
57077/57077 [==============================] - 227s 4ms/step - loss: 4.8151
Epoch 3/200
57077/57077 [==============================] - 225s 4ms/step - loss: 4.8159
Epoch 4/200
57077/57077 [==============================] - 225s 4ms/step - loss: 4.7686
Epoch 5/200
57077/57077 [==============================] - 223s 4ms/step - loss: 4.6836
Epoch 6/200
57077/57077 [==============================] - 224s 4ms/step - loss: 4.7269
Epoch 7/200
57077/57077 [==============================] - 223s 4ms/step - loss: 4.6706
Epoch 8/200
57077/57077 [==============================] - 222s 4ms/step - loss: 4.6640
Epoch 9/200
57077/57077 [==============================] - 222s 4ms/step - loss: 4.5865
Epoch 10/200
57077/57077 [==============================] - 223s 4ms/step - loss: 4.5598
Epoch 11/200
57077/57077 [==============================] - 222s 4ms/step - loss: 4.5495
Epoch 12/200
57077/57077 [====

# 학습, 저장된 모델을 가져와서 MIDI 음원 생성

In [ ]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(location + 'weights-improvement-100-0.2295-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=location + 'test_output.mid')


In [21]:
""" Generate a piano midi file """
#load the notes used to train the model
with open(location + 'data/notes', 'rb') as filepath:
    notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
print(len(pitchnames))
# Get all pitch names
n_vocab = len(set(notes))

network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
model = create_network(normalized_input, n_vocab)
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
create_midi(prediction_output)


158


TypeError: prepare_sequences() takes 2 positional arguments but 3 were given